In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import copy

from collections import defaultdict 

from synthesizer import Synthesizer
from generator import Generator
from algorithm import Algorithm

# warnings.filterwarnings('ignore')
plt.rcParams['figure.dpi'] = 80

### Эксперимент

In [ ]:
# seeds = 5 * np.arange(2, 6) + 111

#### Фиксированные параметры генератора
- series_type = "default"  // обычная линейная регрессия

- dim=10 // размерность сигнала
- low=-10, high=10 // границы весов для получения отклика
- noise_var=1 // дисперсия нормального шума
- workers_num=3 // число различных генераторов
- lower_bound=50, upper_bound=200 // границы длин каждого из кусков, получаемых генератором
- alternating=True // нет двух подряд кусков одного типа
#### Фиксированные параметры алгоритма
- from_start=False // начинаем подсчет всех потерь (и составного эксперта) не с самого начала, а после того как прошли по одному разу учатки каждого из  генераторов

- length=1500 // длина изучаемого участка временного ряда

- a=-40, b=40 // параметры алгоритма, отклики отсеиваем тоже по ним

#### Гиперпараметры эксперимента
- windows = [5, 10, 20] // варьируем длину окна обучения ($\frac12x, 1x, 2x$ соответственно, где $x$  &mdash; dim  сигнала )
- weights_function // функция начальных весов, cравниваем $\dfrac1{x^\alpha}$ для разных $\alpha$ и дефолтную $\dfrac{1}{(x+1)\ln^2(x+1)}$
- alpha_function // функция коэффицента $\alpha$ в Mixing Update, cравниваем $\dfrac1{(1 + x)^\alpha}$ для разных $\alpha$

In [7]:
from hypers import weight_hypers
from hypers import alpha_hypers 

# from experiment import results
# from experiment import interesting_w, interesting_a
# from experiment import windows
# from experiment import df

In [8]:
# df.to_csv("df-2.csv", sep='\t')
df = pd.read_csv("df-2.csv", sep='\t')

In [9]:
df_w = df[df["alpha_function"] == "1 / (x + 1)"].iloc[:, :4]
df_w

,Unnamed: 0,train_window,weight_function,alpha_function
0,8,5,1 / ((x + 1) * (ln(x + 1))^2),1 / (x + 1)
1,2,5,1 / (x^1.05),1 / (x + 1)
6,4,5,1 / (x^1.1),1 / (x + 1)
7,0,5,1 / (x^1.2),1 / (x + 1)
8,1,5,1 / (x^2),1 / (x + 1)
9,17,10,1 / ((x + 1) * (ln(x + 1))^2),1 / (x + 1)
10,11,10,1 / (x^1.05),1 / (x + 1)
15,13,10,1 / (x^1.1),1 / (x + 1)
16,9,10,1 / (x^1.2),1 / (x + 1)
17,10,10,1 / (x^2),1 / (x + 1)


In [10]:
df_a = df[df["weight_function"] == "1 / (x^1.05)"].iloc[:,:4]
df_a

,Unnamed: 0,train_window,weight_function,alpha_function
1,2,5,1 / (x^1.05),1 / (x + 1)
2,7,5,1 / (x^1.05),1 / (x + 1)^1.1
3,6,5,1 / (x^1.05),1 / (x + 1)^1.3
4,3,5,1 / (x^1.05),1 / (x + 1)^1.5
5,5,5,1 / (x^1.05),1 / (x + 1)^2)
10,11,10,1 / (x^1.05),1 / (x + 1)
11,16,10,1 / (x^1.05),1 / (x + 1)^1.1
12,15,10,1 / (x^1.05),1 / (x + 1)^1.3
13,12,10,1 / (x^1.05),1 / (x + 1)^1.5
14,14,10,1 / (x^1.05),1 / (x + 1)^2)


In [11]:
df[["train_window", "mean"]].groupby("train_window").mean()

,mean
train_window,
5,183014.784444
10,142005.222222
20,163716.380000


In [12]:
i = 10
results[i].algo.draw_all(show=["master", "ideal", "theoretical"], show_experts=[], 
                              show_axes=["regret"], height_ratios=[1], 
                             suptitle=f"tw: {results[i].train_window},   wf:  {weight_hypers[results[i].key_w].repr},   af: {alpha_hypers[results[i].key_a].repr},   regret = {results[i].algo.regret:.1f}", 
                             fig_size=(10, 6))

NameError: name 'results' is not defined

In [1]:
for i in df_w.index:
    results[i].algo.draw_all(show=["master", "ideal", "theoretical"], show_experts=[], 
                              show_axes=["regret"], height_ratios=[1], 
                             suptitle=f"tw: {results[i].train_window},   wf:  {weight_hypers[results[i].key_w].repr},  regret = {results[i].algo.regret:.1f}", 
                             fig_size=(6, 4))

NameError: name 'df_w' is not defined

In [9]:
# for i in df_a.index:
#     results[i].algo.draw_all(show=["master", "ideal", "theoretical"], show_experts=[], 
#                               show_axes=["regret"], height_ratios=[1], 
#                                 suptitle=f"tw: {results[i].train_window},   af:  {alpha_hypers[results[i].key_a].repr},  regret = {results[i].algo.regret:.1f}", 
#                              fig_size=(6, 4))

In [10]:
# i = 13
# results[i].algo.draw_all(show=["master", "ideal", "theoretical"], show_experts=[], 
#                               show_axes=["regret"], height_ratios=[1], 
#                              suptitle=f"tw: {results[i].train_window},   wf:  {weight_hypers[results[i].key_w].repr},  regret = {results[i].algo.regret:.1f}", 
#                              fig_size=(6, 4))